### **5.5 - [Practica] Ad Hoc queries with the metadata database**

En el video anterior, hemos descubierto el Local Executor con Postgres pero no tomamos tiempo para echar un vistazo a la metadata database. Vamos a hacer esto. En primer lugar, desde tu editor de código, comprueba que estás en la carpeta airflow- materials/airflow-section-5 y abre el archivo docker-compose-LocalExecutor. yml. Entonces, si recuerdas tenemos un servicio llamado postgres que corresponde a la base de datos Postgres que vamos a utilizar con Airflow. La versión 9.6 está activada como puedes ver aquí y la base de datos está configurada usando las variables de entorno descritas aquí.  Básicamente establecemos el nombre de usuario, la contraseña y el nombre de la base de datos.  Ah, y si te estás preguntando dónde se ejecuta el scheduler ya que sólo tenemos postgres y servicios de web server, en realidad se inicia en el mismo contenedor del web server. Como mejor práctica, debe mantener el web server y el scheduler colocados cuando diseñe su arquitectura Airflow.  Más adelante veremos un ejemplo en el que el scheduler se iniciará en un contenedor diferente, pero en la misma máquina. 

<center><img src="https://i.postimg.cc/2ypkcTKj/a789.png"></center>

Antes de seguir adelante, echemos un vistazo al Dockerfile utilizado en este archivo docker-compose para ejecutar Airflow. Abra el archivo Dockerfile en la carpeta docker/airflow. Usted ya debe estar familiarizado con Docker, pero permítanme darles una visión general del archivo. Se basa en Python 3.7 que es importante porque la versión actual de Airflow ya no soporta Python 2.  Luego tenemos algunos argumentos que podemos establecer en tiempo de construcción (build time) de la imagen Docker cuando escribimos el comando "docker build". Por ejemplo, podemos cambiar la versión de Airflow, añadir dependencias, etc.  A continuación, tenemos las variables de entorno que serán accesibles desde el contenedor Docker.  

<center><img src="https://i.postimg.cc/FHP95hBB/a790.png"></center>

El gran comando "run" instala todo (No sé si dice eso), como dependencias, tools y Airflow. Se exponen los puertos 8080, 5050 y 8793.  

<center><img src="https://i.postimg.cc/xdc99WrT/a791.png"></center>

El primer puerto corresponde al web server mientras que los otros están relacionados con las herramientas utilizadas con el Celery Executor, como Flower y Redis. Lo veremos en el siguiente vídeo.  Por último, se inicia el web server ejecutando el comando que aparece aquí.  

<center><img src="https://i.postimg.cc/SQ8N2Fnn/a792.png"></center>

Una cosa por notar es la ejecución del script entrypoint.sh. Ábrelo desde la carpeta script. 

<center><img src="https://i.postimg.cc/C18YqkL3/a793.png"></center>

Ok ¿qué hace este script? Las primeras líneas con los dos puntos corresponden a las variables de entorno que se definirán dentro del contenedor docker que ejecuta Airflow así como su valor por defecto.  Por ejemplo, si no establecimos desde el archivo docker compose la variable de entorno **`POSTGRES_USER`**, por defecto se establecerá con el valor "airflow". Luego desde la instrucción **export**, se pueden ver las variables de entorno definidas para modificar la configuración de Airflow.  Tenemos el EXECUTOR, si queremos los ejemplos (LOAD_EXAMPLES) o no y demás.  Lo que realmente quiero mostrarte es lo que ocurre cuando se establece la variable **`AIRFLOW__CORE__EXECUTOR`**.  Si te desplazas hacia abajo, podrás ver esta importante condición.  Cuando decimos que queremos el Local Executor, se definen dos parámetros con las variables de entorno.  En primer lugar, la conexión SQL_ALCHEMY_CONN se establece para utilizar la base de datos PostgreSQL con el valor definido anteriormente, como el nombre de usuario, contraseña y nombre db. 

<center><img src="https://i.postimg.cc/Qx3Nc470/a794.png"></center>

Luego, el CELERY__RESULT_BACKEND se establece también con la conexión.  Veremos Celery en el próximo video, pero por ahora, el Celery backend es donde se almacenan los metadatos de los jobs ejecutados.  Así que, repito, este string de aquí, AIRFLOW__CELERY__RESULT_BACKEND, corresponde a la conexión con la base de datos Postgres.  Si echamos un vistazo al archivo de configuración airflow.cfg, y buscamos SQL_ ALCHEMY_CONN, tenemos el mismo string pero esta vez para sqlite como es el caso por defecto.  Nótese que este valor será sobrescrito por la conexión postgres de la variable de entorno AIRFLOW__CORE__SQL_ALCHEMY_CONN.  

<center><img src="https://i.postimg.cc/NjGgzKpX/a795.png"></center>

Bien, el resto del script describe cómo se inician el web server y el scheduler en función de los argumentos dados al ejecutarlo.  Así que hemos visto como se define la conexión a la metadata database con esta imagen Docker, ahora vamos a abrir el terminal. Comprueba que estás en la carpeta airflow -materials/airflow-section-5 con los siguientes archivos. A continuación, iniciamos los contenedores docker con el comando "docker-compose -f docker-compose-LocalExecutor.yml up -d".  Enter.  Comprobamos si los contenedores se están ejecutando con el comando "docker ps".

<center><img src="https://i.postimg.cc/d0pw34vR/a796.png"></center>

Perfecto podemos abrir la interfaz de usuario de Airflow. Desde el navegador web, escribimos localhost:8080 y Enter. Y activamos el toggle del DAG parallel_dag para que tengamos algunos datos en la base de datos.  Espera a que terminen dos o tres DAGRuns y deja de programar (stop scheduling) el DAG.  Ahora, haga clic en "Data Profiling" y "Ad Hoc Query".  

<center><img src="https://i.postimg.cc/fTfzCcjZ/a797.png"></center>

Desde esta página, Airflow le permite consultar las bases de datos que desee seleccionando una conexión de esta lista. Por defecto, Airflow trae un montón de conexiones que, obviamente, no llevan a ninguna parte, ya que no existen.  Son sólo una especie de ejemplos.  

<center><img src="https://i.postimg.cc/wMdzcM17/a798.png"></center>
<center><img src="https://i.postimg.cc/SQWkQ6P8/a799.png"></center>

Para consultar nuestra base de datos Postgres, necesitamos crear una conexión.  Para ello haremos clic en Admin, Conexiones y Crear.  Llamaremos a nuestra conexión "postgres_1".  Luego, en “Conn_Type” seleccionamos Postgres.  Como estamos ejecutando Airflow y la metadata database dentro de contenedores Docker, el host es igual al nombre del servicio dado en el archivo docker compose que es postgres.  El schema es airflow.  A continuación, el nombre de usuario y la contraseña también son iguales a "airflow".  Y, por último, el port se establece en 5432.  Comprueba que no has escrito mal ningún valor aquí y haz clic en guardar.  

<center><img src="https://i.postimg.cc/6qLXHKZz/a800.png"></center>

Muy bien, la conexión está creada podemos volver a la página de ad hoc query. En la lista, haga clic en la conexión que acabamos de crear y ahora podemos empezar a hacer consultas a la base de datos.  

<center><img src="https://i.postimg.cc/m2BRPcxn/a801.png"></center>

Te voy a dar algunas consultas SQL útiles que encontrarás en el archivo sql_queries.pdf bajo la carpeta docs en airflow-section-5.  

<center><img src="https://i.postimg.cc/P5FkV2LG/a802.png"></center>
<center><img src="https://i.postimg.cc/jdcTPstS/a803.png"></center>
<center><img src="https://i.postimg.cc/wjBpkSjc/a804.png"></center>

Empecemos con la primera. Con Postgres, una forma de mostrar las tablas disponibles en la base de datos actual es escribiendo la siguiente consulta:
```
      SELECT 
         * 
      FROM 
         pg_catalog.pg_tables 
      WHERE 
         schemaname != 'pg_catalog' 
      AND schemaname != 'information_schema' 
      AND tableowner = 'airflow';
```
No olvides el punto y coma al final, y pulsa sobre Ejecutar.  

<center><img src="https://i.postimg.cc/1zn1kw68/a805.png"></center>

Como puede ver obtenemos diferentes tablas como connection, variable, dag_run, task_fail, users, dag y demás correspondientes a los metadatos generados por Airflow. Por ejemplo, podemos explorar las instancias de tareas de nuestros DAGs escribiendo:
```
      SELECT * FROM task_instance ;
```
Haga clic en Ejecutar. Y obtenemos la lista de instancias de tareas creadas por el scheduler.  

<center><img src="https://i.postimg.cc/Bvrf4bcM/a806.png"></center>

Tenemos el dag_id que nos dice de donde viene una instancia de tarea, su execution date, start date.  Tenemos el status, si la instancia de tarea ha tenido éxito, ha fallado y así sucesivamente.  El número de intentos y mucho más. Para que te hagas una idea del resto del curso, las columnas hostname y queue tendrán sentido con el Celery Executor y descubrirás pools y priority_weights en los conceptos avanzados de Airflow.  Así que sigue con el curso que viene lo mejor.  Hemos visto las instancias de tareas, podemos echar un vistazo a los DagRuns también:
```
      SELECT * FROM dag_run ;
```
entonces ejecutamos la consulta y obtenemos la lista de DagRuns.  

<center><img src="https://i.postimg.cc/MT12Tw9x/a807.png"></center>

Ahora digamos que nos gustaría tener el número total de tareas completadas.  Podríamos escribir:
```
      SELECT COUNT(1) 
      FROM task_instance 
      WHERE 
      state IS NOT NULL 
      AND state NOT IN ('scheduled', 'queued');
```
Pulsamos ejecutar, y obtenemos el número de tareas completadas.  

<center><img src="https://i.postimg.cc/7hDvJRRz/a808.png"></center>

Por último, esta vez nos gustaría tener el número de tareas iniciadas por minuto durante la última semana. Para ello, podemos teclear:
```
      SELECT 
      date_trunc(‘minute’, start_date) AS d, 
      count(1) 
      FROM task_instance 
      GROUP BY d 
      ORDER BY 1 DESC 
      LIMIT 24*7
```
Ahora pulsamos en Ejecutar, y obtenemos el resultado.  

<center><img src="https://i.postimg.cc/sxJkYcFN/a809.png"></center>

Muy bien, no dude en hacer sus propias consultas para explorar los datos proporcionados por Airflow.  Esta función está muy bien, pero ¿qué pasa con la seguridad? Como puede adivinar, la posibilidad de extraer datos directamente de Airflow es bastante insegura y no es deseable para la mayoría de las empresas.  Bueno, puede desactivar esta función cambiando el parámetro **`secure_mode`** en **`airflow.cfg`**.  Vamos a hacer esto.  Vuelva a su editor de código, abra el archivo airflow.cfg en la carpeta mnt/airflow. **`Busque el parámetro secure_mode y cambie el valor de False a True`**. Guarde el archivo y abra su terminal.  

<center><img src="https://i.postimg.cc/nVYffHsG/a810.png"></center>

Reinicie los contenedores docker escribiendo "docker-compose -f docker-compose-LocalExecutor.yml down", ok, luego "docker-compose -f docker-compose-LocalExecutor.yml up -d".

<center><img src="https://i.postimg.cc/gkzFYf8F/a811.png"></center>

Muy bien, volvamos a la interfaz de usuario de Airflow. Haga clic en Dags. Y ahora, si haces clic en "Data Profiling" ya no puedes acceder a la página "Ad Hoc Query". 

<center><img src="https://i.postimg.cc/QxfLV0JB/a812.png"></center>

Como mejor práctica, le recomiendo encarecidamente que mantenga el parámetro **`secure_mode en True`**, ya que Airflow no debe ser utilizado como un explorador de base de datos. Hay otras herramientas para hacer eso, así que mantenga Airflow como un orquestador y nada más.  Y, por cierto, este parámetro se establecerá en true en la futura versión de Airflow.  Muy bien eso es todo por este video.  Antes de pasar al siguiente, vuelve a tu terminal y detén el contenedor docker con el mismo comando que hicimos anteriormente.  

<center><img src="https://i.postimg.cc/P5JgDYSS/a813.png"></center>